In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np

sys.path.insert(0, '../../../batchflow')
sys.path.insert(0, '../../')

from seismiqb import SeismicDataset, Horizon

In [ ]:
GEOMETRY_PATH = 'cube.blosc'
HORIZONS_PATH = 'horizons/*'

dataset = SeismicDataset({GEOMETRY_PATH: {'horizons': HORIZONS_PATH}}, interpolate=True)
print(dataset)

In [ ]:
field = dataset[0]
horizon = field.horizons[0]

## Field attributes

Use `mode='histogram'` to plot histograms for desired field attributes.

In [ ]:
field.show(
    attributes=[
        'snr',
        'horizons:*/depths',
        'horizons:*/amplitudes',
        'horizons:*/instant_amplitudes'
    ],
    mode='histogram',
    combine='separate',
    ncols=2
)

In [ ]:
field.show(
    attributes=[f'horizons:{i}/depths' for i in range(len(field.horizons))],
    mode='histogram',
    title_pattern='Depths distribution of {label_name}',
    suptitle_size=20, title_size=15, title_wrap_width=100, title_wrap_delimiter=','
)

## Metrics

Some functions, like `horizon.compare` already use `mode='histogram'` under the hood.

In [ ]:
def gkern(size, sigma):
    x = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
    gauss = np.exp(-0.5 * np.square(x) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

def add_horizon_anomalies(horizon):
    shifts = np.zeros(horizon.matrix.shape, dtype=np.int32)

    for _ in range(np.random.randint(10, 20)):
        size = np.random.randint(10, 50)
        kernel = gkern(size, size * 0.3)
        kernel = kernel / kernel.max() * np.random.randint(5, 10)
        kernel = np.repeat(kernel, np.random.randint(1,4), axis=np.random.randint(2))
        kernel *= np.random.choice([-1, 1])

        i, x, h = horizon.points[np.random.choice(len(horizon))]
        i_start = i - kernel.shape[0] // 2
        i_stop = i_start + kernel.shape[0]
        x_start = x - kernel.shape[1] // 2
        x_stop = x_start + kernel.shape[1]

        if (i_start < 0) or (x_start < 0) or (i_stop > shifts.shape[0]) or (x_stop > shifts.shape[1]):
            continue

        shifts[i_start : i_stop, x_start : x_stop] += kernel.astype(np.int32)

    shifts[horizon.matrix < 0] = horizon.FILL_VALUE

    shifted_matrix = horizon.matrix + shifts
    shifted = Horizon(shifted_matrix, horizon.field,
                      name=f"shifted {horizon.name}", i_min=horizon.i_min, x_min=horizon.x_min)
    
    return shifted

Note that additional arguments for histogram might be provided with `hist_` prefix.

In [ ]:
anomalous = add_horizon_anomalies(horizon)

One can provide parameters for those plots histograms via `hist_kwargs`.

In [ ]:
_ = horizon.compare(
    anomalous,
    printer=lambda _: None,
    ignore_zeros=True,
    hist_kwargs=
    {
        'color': 'lightcoral',
        'histogram_log': True,
        'histogram_cumulative': -1,
        'histogram_histtype': 'step',
        'histogram_linewidth': 3,
        'histogram_linestyle': '--',
        'title': 'Cumulative histogram of horizon depths differences'
    }
)